In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load Ground Truth

In [2]:
# load json test_data_annotations.json
with open('test_data_annotations.json') as f:
    ground_truth = json.load(f)
ground_truth

{'0': ['3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes'],
 '1': ['3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d_printed_clothes',
  '3d

# Load Predictions

In [3]:
# load predictions.jsonl from current folder
preds = []
with open("predictions.jsonl") as f:
    lines = f.readlines()
    for idx, line in enumerate(lines):
        try:
            pred =  json.loads(line)
            preds.append(pred)
        except:
            print("Failed to load line")
preds = {list(p.keys())[0]:p[list(p.keys())[0]] for p in preds}
new_rows = []
for article_id, section_predictions in preds.items():
    for section_pred in section_predictions:
        new_row = {
            "article_id": article_id,
            "section_id": section_pred["section_id"],
            "y_hat": section_pred["category"],
            "text": section_pred["text"],
            "reasoning": section_pred["reasoning"],
            "explanation": section_pred["explanation"],
        }
        new_rows.append(new_row)
predictions = pd.DataFrame(new_rows)
# sort by article id and section id
predictions = predictions.sort_values(by=["article_id", "section_id"]).reset_index(drop=True)
predictions.head(n=10)

article_id  section_id                y_hat  \
0          0           0   3d_printed_apparel   
1          0           1   3d_printed_apparel   
2          0           7   3d_printed_apparel   
3          0           8   3d_printed_apparel   
4          0           9   3d_printed_apparel   
5          0          11  sustainable_fabrics   
6          0          12   3d_printed_apparel   
7          0          13   3d_printed_apparel   
8          0          14   3d_printed_apparel   
9          0          15   3d_printed_apparel   

                                                text  \
0   When the fashion trio threeASFOUR debuted its...   
1  When the fashion trio threeASFOUR debuted its ...   
2  The trio at its helm — Gabi Asfour and his des...   
3  They wanted to do the opposite: stretch clothi...   
4  They dreamed of 3D-printing textiles that were...   
5  At a moment when Silicon Valley’s elite are bu...   
6  But where hoarding cans of Doomsday beans is w...   
7  Thousands of years of refinement have allowed ...   
8  But that hasn’t stopped the team behind threeA...   
9  “With fashion there is the potential to contro...   

                                           reasoning  \
0  The section clearly discusses the debut collec...   
1  The section discusses the debut of a collectio...   
2  The article segment clearly discusses the crea...   
3  The section describes the ambition to 3D-print...   
4  The article segment discusses the dream of cre...   
5  The article section discusses the efficiency a...   
6  The section discusses the challenges and innov...   
7  The article segment emphasizes the potential t...   
8  The section discusses the potential to control...   
9  The article segment discusses the potential of...   

                                         explanation  
0  {'1': {'prediction_1': {'category': '3d_printe...  
1  {'0': {'prediction_1': {'category': 'unsure', ...  
2  {'1': {'prediction_1': {'category': '3d_printe...  
3  {'1': {'prediction_1': {'category': '3d_printe...  
4  {'1': {'prediction_1': {'category': '3d_printe...  
5  {'0': {'prediction_1': {'category': 'sustainab...  
6  {'1': {'prediction_1': {'category': '3d_printe...  
7  {'1': {'prediction_1': {'category': '3d_printe...  
8  {'0': {'prediction_1': {'category': '3d_printe...  
9  {'1': {'prediction_1': {'category': '3d_printe...

## Add Ground Truth Labels to Predictions

In [4]:
dfs = []
for article_id, true_labels in ground_truth.items():
    predictions_sub = predictions[predictions["article_id"] == article_id].copy()
    predictions_sub.loc[:, "y"] = true_labels
    # get article level predictions as majority vote of section level predictions
    predictions_sub.loc[:, "article_level_prediction"] = predictions_sub["y"].mode().values[0]
    dfs.append(predictions_sub)
predictions = pd.concat(dfs)
# rename category in true labels
predictions["y"] = predictions["y"].apply(lambda x: "3d_printed_apparel" if x == "3d_printed_clothes" else x)
# change category columns to category type
predictions["y_hat"] = predictions["y_hat"].astype("category")
predictions["y"] = predictions["y"].astype("category")
predictions["article_level_prediction"] = predictions["article_level_prediction"].astype("category")
predictions.head(n=10)

article_id  section_id                y_hat  \
0          0           0   3d_printed_apparel   
1          0           1   3d_printed_apparel   
2          0           7   3d_printed_apparel   
3          0           8   3d_printed_apparel   
4          0           9   3d_printed_apparel   
5          0          11  sustainable_fabrics   
6          0          12   3d_printed_apparel   
7          0          13   3d_printed_apparel   
8          0          14   3d_printed_apparel   
9          0          15   3d_printed_apparel   

                                                text  \
0   When the fashion trio threeASFOUR debuted its...   
1  When the fashion trio threeASFOUR debuted its ...   
2  The trio at its helm — Gabi Asfour and his des...   
3  They wanted to do the opposite: stretch clothi...   
4  They dreamed of 3D-printing textiles that were...   
5  At a moment when Silicon Valley’s elite are bu...   
6  But where hoarding cans of Doomsday beans is w...   
7  Thousands of years of refinement have allowed ...   
8  But that hasn’t stopped the team behind threeA...   
9  “With fashion there is the potential to contro...   

                                           reasoning  \
0  The section clearly discusses the debut collec...   
1  The section discusses the debut of a collectio...   
2  The article segment clearly discusses the crea...   
3  The section describes the ambition to 3D-print...   
4  The article segment discusses the dream of cre...   
5  The article section discusses the efficiency a...   
6  The section discusses the challenges and innov...   
7  The article segment emphasizes the potential t...   
8  The section discusses the potential to control...   
9  The article segment discusses the potential of...   

                                         explanation                   y  \
0  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
1  {'0': {'prediction_1': {'category': 'unsure', ...  3d_printed_apparel   
2  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
3  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
4  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
5  {'0': {'prediction_1': {'category': 'sustainab...  3d_printed_apparel   
6  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
7  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
8  {'0': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   
9  {'1': {'prediction_1': {'category': '3d_printe...  3d_printed_apparel   

  article_level_prediction  
0       3d_printed_clothes  
1       3d_printed_clothes  
2       3d_printed_clothes  
3       3d_printed_clothes  
4       3d_printed_clothes  
5       3d_printed_clothes  
6       3d_printed_clothes  
7       3d_printed_clothes  
8       3d_printed_clothes  
9       3d_printed_clothes

# Evaluate Results
## Predicted Categories that don't exist (should not be possible)

In [5]:
true_categories = predictions["y"].unique()
unique_non_existing_predictions = predictions[~predictions["y_hat"].isin(true_categories)]["y_hat"].unique()
for cat in unique_non_existing_predictions:
    print(cat)

3d_printed_fashion
3d_printing
none
3d_printed_food
public_transport
autonomous transport
smart_collaboration_tools
smart_energy_grids
autonomous_driving
low_impact_living
low_energy_demand
sustainable_water_production_treatment
sustainable_forestry
sustainable_transport
sharing_economy
sustainable_agriculture_production
carbon_price
chemicals_production
solar_energy
sustainable_washing
dematerialisation
sustainable_construction_materials
nanotechnology
drones
modular_design
unsure
scope_for_avoided_emissions
air_quality_pollution_control
health_outcomes_measurement
videoconferencing
teleworking
telemedicine
blockchain
universal_basic_income
digital_solutions_to_regulate_food_intake_and_nutrition
healthcare_technology
electric_vehicles
circular_economy
electric car
electric_vehicles_vs_fuel_efficient_used_cars
electric_vehicle_incentives
led
energy_storage
urban_farming
daylight_architecture
sustainable_energy
sustainable_fabric
e_learning
sustainable_energy_production
sustainable_ship

In [6]:
true_categories

['3d_printed_apparel', 'autonomous_transport', 'biking', 'capsule_wardrobe', 'car_sharing', ..., 'rent_apparel', 'sustainable_fabrics', 'organic_food', 'smart_food_management_kitchen_fridges_freezers', 'artificial_intelligence']
Length: 17
Categories (17, object): ['3d_printed_apparel', 'artificial_intelligence', 'autonomous_transport', 'biking', ..., 'rent_apparel', 'shipping', 'smart_food_management_kitchen_fridges_freezers', 'sustainable_fabrics']

In [7]:
print("Percentage of non-existing categories in predictions: {:.2f}%".format(100 * len(predictions[~predictions["y_hat"].isin(true_categories)]) / len(predictions)))
predictions = predictions[~predictions["y_hat"].isin(unique_non_existing_predictions)]

Percentage of non-existing categories in predictions: 18.89%


In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(predictions["y"], predictions["y_hat"], labels=true_categories)
cm = pd.DataFrame(cm, index=true_categories, columns=true_categories)

In [9]:
from sklearn.metrics import classification_report
classification_report = classification_report(predictions["y"], predictions["y_hat"], target_names=true_categories, output_dict=True)
classification_report = pd.DataFrame(classification_report).T
classification_report

precision    recall  f1-score  \
3d_printed_apparel                               0.971698  0.841307  0.901814   
autonomous_transport                             0.923567  1.000000  0.960265   
biking                                           0.873909  0.982552  0.925051   
capsule_wardrobe                                 0.977901  1.000000  0.988827   
car_sharing                                      1.000000  0.973118  0.986376   
cement_production                                1.000000  0.961415  0.980328   
drone                                            1.000000  1.000000  1.000000   
e_health                                         1.000000  0.677966  0.808081   
electric_car                                     1.000000  0.731183  0.844720   
fiber_optic_cables                               0.999178  0.998357  0.998767   
shipping                                         1.000000  0.970085  0.984816   
clothes_designed_for_a_circular_economy          1.000000  0.919540  0.958084   
rent_apparel                                     1.000000  0.947761  0.973180   
sustainable_fabrics                              0.982249  0.927374  0.954023   
organic_food                                     0.908078  0.942197  0.924823   
smart_food_management_kitchen_fridges_freezers   0.982587  0.965770  0.974106   
artificial_intelligence                          0.773070  0.997570  0.871088   
accuracy                                         0.939434  0.939434  0.939434   
macro avg                                        0.964249  0.931541  0.943197   
weighted avg                                     0.948379  0.939434  0.939201   

                                                    support  
3d_printed_apparel                               857.000000  
autonomous_transport                             145.000000  
biking                                           917.000000  
capsule_wardrobe                                 354.000000  
car_sharing                                      372.000000  
cement_production                                311.000000  
drone                                            154.000000  
e_health                                         177.000000  
electric_car                                     465.000000  
fiber_optic_cables                              1217.000000  
shipping                                         234.000000  
clothes_designed_for_a_circular_economy          435.000000  
rent_apparel                                     134.000000  
sustainable_fabrics                              179.000000  
organic_food                                     346.000000  
smart_food_management_kitchen_fridges_freezers   409.000000  
artificial_intelligence                          823.000000  
accuracy                                           0.939434  
macro avg                                       7529.000000  
weighted avg                                    7529.000000

In [10]:

plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True") 
plt.title("Confusion Matrix")
plt.show()